## Planteamos el problema

Queremos obtener la cartera óptima de un conjunto de 70.000 activos. 

Vamos a intentar resolver el problema usando algoritmos genéticos. Para lo que tendremos que resolver distintos puntos:

1. Los inversores (cromosomas) deben poder generar carteras con un número variable de activos. Estableceremos el mínimo en 1 activo y 20 en el máximo. El número de activos debe poder modificarse entre generaciones, sin caer en extremos (que siempre salgan carteras de 1 activo o de 20). Debemos pensar un sistema coherente, para resolver este punto.

2. El método de selección de padres no parece, a priori, problemático, pero el cruce y la mutación sí. ¿Cómo va a heredar el hijo la información genética de los padres? Aquí tenemos un triple problema a resolver:

    * ¿Cual es el número de activos que hereda un hijo de dos padres? Si un padre tiene 5 activos y el otro 7 (distintos entre sí), ¿cuantos activos tendrá el hijo? Debemos pensar en el punto primero. El hijo no puede tener un número de activos creciente en cada generación. El sistema tiene que ser dinámico, creciente y decreciente, respetando los límites de 1 y 20.
    
    * ¿Qué porcentaje de inversión hereda cada activo para que sume 100%? Debemos permitir la construcción de carteras donde unos pocos activos (1 o 2) se lleven un porcentaje muy elevado del capital disponible, para explorar todas las soluciones. Reescalar los pesos no es una solución aceptable.
    
    * La evolución, en este problema no puede limitarse a selección y cruzamiento o caeremos rápidamente en un mínimo local. Debemos encontrar un sistema que equilibre la selección con la exploración, si queremos encontrar la cartera óptima. 

In [1]:
import pandas as pd
import numpy as np
import random

## Cargamos los datos y los homogeneizamos

In [2]:
def cargar_ficheros():
    
    onlyfiles = [f for f in listdir('Datos') if isfile(join('Datos', f))]

    datos = []

    for fichero in onlyfiles:

        # print(join('Datos', fichero))    
        df = pd.read_csv(join('Datos', fichero))    
        datos.append(df)

    datos = pd.concat(datos)
    return datos

In [3]:
def comprobar_homogeneizacion(datos):

    # Comprobamos si todos los activos tienen cotizaciones todos los días

    long_fechas_unicas = datos.date.unique().shape[0]
    long_isin_unicos = datos.iloc[:,2].unique().shape[0]

    if (long_fechas_unicas * long_isin_unicos == datos.shape[1]) == True:
        print("Los activos están homogeneizados")
        return True
    else:
        print("Los activos NO están homogeneizados")
        return False    
    

In [4]:
def extraccion_datos(isin, datos):
    
    # Localizo los datos
    df_intermedio = datos.loc[datos["isin"]==isin,:]

    # Genero un nuevo DF solo con los datos que nos interesan:
    # Fecha como index y nav, siendo el isin el nombre de la columna.
    nuevo_df = pd.DataFrame(df_intermedio, columns=["date","nav"])
    
    # Hay isins con fechas repetidas
    nuevo_df = nuevo_df.groupby('date').first().reset_index()      
    
    nuevo_df.index = nuevo_df.date  
    nuevo_df.index = pd.to_datetime(nuevo_df.index)    
    del nuevo_df["date"]
    nuevo_df.columns = [isin]
    
    return nuevo_df

In [5]:
def homogeizar(datos):

    lista_isin = datos.iloc[:,2].unique()

    fechas_posibles = datos.date.unique()
    fechas_posibles.sort()
    fechas_posibles = pd.date_range(fechas_posibles[0], fechas_posibles[-1], freq=BDay())
    datos_ordenados = pd.DataFrame(columns=[""], index = fechas_posibles) 

    for isin in lista_isin:

        print(isin)
        isin_extraido = extraccion_datos(isin, datos)

        # La gran mayoría tiene más de 200 días, pero hay varios con menos de 100 datos
        # Debemos filtrar los activos para poder trabajar

        if (isin_extraido.shape[0] > fechas_posibles.shape[0]*0.7):

            datos_ordenados = pd.concat([datos_ordenados, isin_extraido], axis=1) # alternativa datos_ordenados = datos_ordenados.join(isin_extraido)

    datos_ordenados = datos_ordenados.drop(columns='')

    # Encontramos las filas donde un % elevado son Nan y las eliminamos
    dias_eliminar = ~(datos_ordenados.isna().sum(axis=1) > datos_ordenados.shape[1]*0.9)
    datos_ordenados = datos_ordenados.loc[dias_eliminar,:]

    # Completamos los Nan con los datos del día anterior
    datos_ordenados = datos_ordenados.fillna(method='ffill')

    # Aquellos valores que no tienen un primer día cotizado, el método anterior no sirve
    datos_ordenados = datos_ordenados.fillna(method='bfill')

    datos_ordenados.to_pickle("datos_ordenados.pkl")     
    
    return datos_ordenados

In [6]:
def cargar_datos():

    try:
        datos_ordenados = pd.read_pickle("datos_ordenados.pkl")

    except:
        datos = cargar_ficheros()
        comprobacion = comprobar_homogeneizacion(datos)

        if comprobacion == False:
            datos_ordenados = homogeizar(datos)
            
    return datos_ordenados

## Generamos la población inicial

In [7]:
def asignar_pesos(posicion_cartera, num_sim = 100):

    '''
    El cómo asignamos pesos a los activos es la primera decisión importante a tomar en el algoritmo. El problema es el siguiente:
    ¿Cómo hago que todos los activos del vector reciban un peso y, a la vez, permito que uno o dos activos se puedan llevar un % muy alto o, incluso, el 100%?
    Pensemos en las posibilidades existentes:

        1) Asignamos un peso aleatorio a cada activo y reescalamos los pesos a 1. Bueno: todos los activos seleccionados tendrán peso. Malo: ningún activo tendrá un % alto en carteras de más de 3 activos.
        2) Recorremos aleatoriamente los activos seleccionados, y les asignamos el % que salga. Al llegar a 100% paramos. Bueno: Los activos pueden tener pesos muy altos en la cartera. Malo: muchos activos se quedarán sin peso.
        3) Asignamos un % aleatorio al primer activo y reescalamos el resto hasta 1. Parece una mala idea...
        
    La segunda solución parece la mejor. Pero tras estudiarla asigna peso a 3 - 5 activos como máximo en el 99% de los casos. Por lo que carteras de hasta 20 activos no van a generarse.
    La solución es evidente. ¿Y si en vez de sacar solo un vector de pesos, sacamos 100 vectores de pesos distintos para estos activos?
    De esta manera, cada inversor tendrá una composición de cartera distinta (posiciones de activos) y 100 configuraciones posibles. 
    La necesidad de explorar configuraciones decrece enormemente aplicando esta solución.
    '''

    pesos_cartera = np.empty([num_sim, len(posicion_cartera)])

    for sim in range(num_sim):

        # Obtenemos el órden en el que vamos a asignar los pesos a los activos
        posiciones_aleatorias = random.sample(range(len(posicion_cartera)), len(posicion_cartera))
        peso_total = 0

        for pos in posiciones_aleatorias:

            if peso_total < 1:

                aleatorio = np.random.random()        

                pesos_cartera[sim, pos] = aleatorio
                peso_total += pesos_cartera[sim, pos]

                if peso_total >= 1:

                    exceso = peso_total - 1
                    pesos_cartera[sim, pos] = pesos_cartera[sim, pos] - exceso
                    peso_total = 1

            else:

                # De esta manera nos aseguramos que el peso es 0. Dado que la generación a ceros del array no es exacta.
                pesos_cartera[sim, pos] = 0

        # Si la cartera tiene muy pocos activos (1 o 2), existe la posibilidad de que no hayamos sumado 100. Aquí sí reescalamos.
        if pesos_cartera[sim,:].sum() < 1:

            pesos_cartera[sim,:] = pesos_cartera[sim,:] / pesos_cartera[sim,:].sum()

    return pesos_cartera

In [8]:
def generar_poblacion_inicial(datos_ordenados, num_act_min = 2, num_act_max = 20):

    # Generamos la población inicial en función de la siguiente relación NCro=Ngen*50/(5+Ngen)

    num_inversores = int(np.ceil((datos_ordenados.shape[1] * 50) / (5 + datos_ordenados.shape[1])))
    num_activos_gen_inicial = np.random.randint(low=num_act_min, high=num_act_max, size=num_inversores)
    lista_posiciones = list()
    
    for inv in range(num_inversores):

        # Cada inversor se compone de posiciones (qué activos) y pesos (% invertido en cada activo)
        # Vamos a centrarnos en las posiciones, dado que los pesos lo resolveremos en la función de fitness.
        
        posicion_cartera = np.random.randint(low=0, high= (datos_ordenados.shape[1]-1), size=num_activos_gen_inicial[inv])
        lista_posiciones.append(posicion_cartera)
           
    return lista_posiciones


## Calculamos la función de fitness

In [9]:
def calcular_fitness(datos_ordenados, posiciones):
    
    # La función objetivo del AG es maximizar la eficiencia de la cartera (su relación rentabilidad / riesgo)
    
    datos = datos_ordenados.iloc[:,list(posiciones)]

    # Calculamos la rentabilidad de los activos
    rent_activos = np.log(datos).diff() 
    rent_activos = rent_activos.iloc[1:,:]

    # Calculamos la matriz de correlaciones
    matriz_correlaciones = rent_activos.corr()

    # Calculamos la matriz de varianzas / covarianzas
    matriz_covarianzas = rent_activos.cov()
    matriz_covarianzas = matriz_covarianzas.to_numpy(dtype='float') 

    # Calculamos la rentabilidad diaria del periodo para cada activo: LN(precio final/ precio inicial)/nº de datos
    precio_inicial = datos.iloc[0,:]
    precio_final = datos.iloc[-1,:]
    rentabilidad_diaria = np.log(precio_final / precio_inicial) / datos.shape[0]

    # Sacamos la matriz de pesos para cada cartera
    matriz_pesos = asignar_pesos(posiciones)
    
    # Calculamos la rentabilidad de la cartera en función de los pesos 
    auxiliar = rentabilidad_diaria.values * matriz_pesos
    rentabilidad_cartera = auxiliar.sum(axis=1)

    # Calculamos el riesgo de la cartera (desviación), en función de los pesos
    auxiliar = np.dot(matriz_pesos, matriz_covarianzas) # Reutilizo la matriz auxiliar con otro propósito para no sobrecargar la RAM
    riesgo_cartera = pow((auxiliar * matriz_pesos).sum(axis=1), 0.5)

    # Calculamos la eficiencia de la cartera (pendiente), en función de los pesos
    # Existen posiciones que van a dar error, 0/0. Las localizo y hago que 0/1 = 0
    posiciones_0entre0 = (rentabilidad_cartera==0) * (riesgo_cartera==0)    
    riesgo_cartera[posiciones_0entre0] = 0.0000001
    eficiencia_cartera = rentabilidad_cartera / riesgo_cartera
    
    # Localizo la cartera de mayor eficiencia y sus pesos correspondientes
    posicion_cartera_eficiente = np.where(eficiencia_cartera == eficiencia_cartera.max())
    eficiencia_cartera = eficiencia_cartera.max()
    pesos_cartera_eficiente = matriz_pesos[posicion_cartera_eficiente,:]
        
    return eficiencia_cartera, pesos_cartera_eficiente

## Selección de padres

In [10]:
def seleccion_padres(lista_fitness):

    # Normalizamos con el método Min - Max para que los valores de la eficiencia estén entre 0 y 1
    fitness_normalizado = np.asarray(lista_fitness)
    fitness_normalizado = (fitness_normalizado - fitness_normalizado.min()) / (fitness_normalizado.max() - fitness_normalizado.min())

    # El órden en el que se van a evaluar los cromosomas para ser seleccionados debe ser aleatorio
    orden_aleatorio = random.sample(range(len(fitness_normalizado)), len(fitness_normalizado))

    padres_seleccionados = list()
    for candidato in orden_aleatorio:

        # Sacamos un número entre 0 y 1 y lo comparamos con el valor de fitness_normalizado del candidato
        # El valor de fitness actúa como un umbral
        # Si el aleatorio es menor al fitness, el candidato se convierte en uno de los dos padres
        # De esta manera todos los candidatos tienen posibilidades de ser padre
        # los que tienen mayor valor de fitness, son los que tienen más probabilidad de tener descendencia.

        aleatorio = np.random.random() 

        if fitness_normalizado[candidato] > aleatorio:

            padres_seleccionados.append(candidato)

            if len(padres_seleccionados) == 2:

                break
   
    return padres_seleccionados

## Cruzamiento y mutación

In [11]:
def cruzamiento(datos_ordenados, lista_posiciones, lista_pesos_carteras_fitness, padres_seleccionados, umbral = 0.4, num_act_max = 20):

    '''
    Las técnicas habituales de cruzamiento en algoritmos genéticos (N-puntos, segmentación, uniforme y aleatorio) no parecen apropiadas para resolver este problema
    Queremos generar un sitema que cruce la información genética de los padres, permitiendo que el tamaño (número de activos) del hijo difiera del de los padres (sin tener que ser forzosamente mayor)
    EL sistema de cruzamiento debe centrarse en la selección de activos y no en los pesos. Dado que los pesos serán calculados durante la función de fitness
    Hemos usado en las funciones previas dos de los elementos más importantes para resolver el problema: pesos y eficiencia (relación rentabilidad riesgo)
    Para hacer el cruzamiento, debemos usar el último elemento importante que queda: la correlación (medida normalizada de la covarianza)

    Pasos a dar

        1) Nos quedamos con los activos de los padres que tengan peso (el resto los descartamos)
        2) Comprobamos que no existen activos repetidos
        3) Calculamos la matriz de correlaciones de los activos
        4) De aquel conjunto de activos que sean muy correlados, nos quedaremos únicamente con el activo más eficiente (rentabilidad / riesgo)
        5) Aplicamos la función de mutación para resolver el problema de la exploración (ver documentación de la función)
        6) Comprobamos que el número de activos no supera el máximo permitido y devolveremos las posiciones de los activos que hereda el hijo

    '''

    # Nos quedamos con los activos de los padres que tengan peso (el resto los descartamos)
    posiciones_activos_con_peso_papa = np.where(lista_pesos_carteras_fitness[padres_seleccionados[0]] != 0)
    posiciones_activos_con_peso_mama = np.where(lista_pesos_carteras_fitness[padres_seleccionados[1]] != 0)

    activos_con_peso_papa = lista_posiciones[padres_seleccionados[0]][posiciones_activos_con_peso_papa[2]]
    activos_con_peso_mama = lista_posiciones[padres_seleccionados[1]][posiciones_activos_con_peso_mama[2]]

    # Comprobamos que no existen activos repetidos
    activos_con_peso = np.unique(np.append(activos_con_peso_papa, activos_con_peso_mama))
    activos_con_peso

    # Calculamos la rentabilidad y riesgo para el periodo de los activos
    datos = datos_ordenados.iloc[:,activos_con_peso]

    precio_inicial = datos.iloc[0,:]
    precio_final = datos.iloc[-1,:]
    rent_global_activos = np.log(precio_final / precio_inicial) 

    riesgo_global_activos = np.std(datos, axis=0)

    eficiencia_global_activos = rent_global_activos / riesgo_global_activos

    # Calculamos la matriz de correlaciones de los activos
    rent_activos = np.log(datos).diff() 
    rent_activos = rent_activos.iloc[1:,:]

    matriz_correlaciones = rent_activos.corr()

    # Calculamos un umbral, a partir del cual descartaremos activos por ser demasiado similares
    # Cuanto más descorrelados estén los activos, más eficiente será la cartera, por lo que premiamos la descorrelación
    # De aquel conjunto de activos que sean muy correlados, nos quedaremos únicamente con el activo más eficiente (rentabilidad / riesgo)

    relaciones_a_analizar = matriz_correlaciones[np.logical_and(matriz_correlaciones > umbral, matriz_correlaciones != 1)]
    relaciones_a_analizar = np.where(matriz_correlaciones == relaciones_a_analizar)

    if len(relaciones_a_analizar[0]) > 0:

        # Recorro todas las relaciones a analizar y fabrico una lista de los activos a quitar (soy consciente de que las relaciones están duplicadas)
        activos_a_eliminar = list()

        for pos in range(len(relaciones_a_analizar[0])):

            if eficiencia_global_activos[relaciones_a_analizar[0][pos]] > eficiencia_global_activos[relaciones_a_analizar[1][pos]]:

                activos_a_eliminar.append(relaciones_a_analizar[1][pos])

            else:

                activos_a_eliminar.append(relaciones_a_analizar[0][pos])

        activos_a_eliminar = list(set(activos_a_eliminar))    
        activos_con_peso = np.delete(activos_con_peso, activos_a_eliminar)

    # Aplicamos la función de mutación para resolver el problema de la exploración
    nuevos_activos = mutacion(datos_ordenados, activos_con_peso, num_act_max = 20)

    activos_heredados = np.unique(np.append(activos_con_peso, nuevos_activos))

    # Comprobamos que el número de activos no supera el máximo permitido y devolveremos las posiciones de los activos que hereda el hijo
    # Tal y como está programado no debería de ocurrir, pero por si acaso. Si ocurriese mucho, habría que modificar el umbral
    if len(activos_heredados) > num_act_max:

        seleccion_aleatoria = random.sample(range(len(activos_heredados)), len(activos_heredados))[0:20]
        activos_heredados = activos_heredados[seleccion_aleatoria]    

    return activos_heredados

In [12]:
def mutacion(datos_ordenados, activos_con_peso, num_act_max = 20):
    
    '''
    La evolución, en este problema no puede limitarse a selección y cruzamiento o caeremos rápidamente en un mínimo local
    Debemos encontrar un sistema que equilibre la selección con la exploración, si queremos encontrar la cartera óptima
    Una solución sería que un % del genoma se herede y el resto se dedique a la exploración
    Un 20% sobre la cantidad máxima de activos parece un umbral máximo aceptable para combinar las tareas de evolución y exploración
    Sacaremos un número aleatorio entre 1 y 4, que serán los nuevos activos para explorar el conjunto de soluciones posibles

    '''

    num_nuevos_activos = np.random.randint(low=1, high=4, size=1)

    nuevos_activos = np.random.randint(low=0, high=datos_ordenados.shape[1]-1, size=num_nuevos_activos)

    return nuevos_activos

## Reemplazo generacional

In [15]:
def reemplazo_generacional(datos_ordenados, lista_posiciones, lista_fitness, lista_pesos_carteras_fitness, activos_cartera_mejor_individuo):

    '''
    La estrategia de reemplazo generacional que vamos a utilizar es reemplazo total. 
    La generación de hijos reemplaza a sus padres, a excepción del mejor de la generación anterior.
    '''
    
    # Rellenamos la matriz de descendencia, la cual será una lista con la composición de activos de cada individuo
    matriz_descendencia = list()
    matriz_descendencia.append(activos_cartera_mejor_individuo)

    for hijo in range(len(lista_posiciones)-1):

        # Seleccionamos los padres
        padres_seleccionados = seleccion_padres(lista_fitness)

        # Aplicamos el cruzamiento y la mutación
        activos_heredados = cruzamiento(datos_ordenados, lista_posiciones, lista_pesos_carteras_fitness, padres_seleccionados, umbral = 0.4, num_act_max = 20)

        # Añadimo el nuevo hijo a la matriz de descendencia
        matriz_descendencia.append(activos_heredados)

    # Para que el algoritmo pueda ejecutarse de manera circular, la matriz de descendencia se ha de llamar lista_posiciones
    lista_posiciones = matriz_descendencia

    return lista_posiciones, fitness_mejor_individuo

## Ejecutamos el algoritmo genético

In [33]:
datos_ordenados = cargar_datos()

# Generamos la población inicial
lista_posiciones = generar_poblacion_inicial(datos_ordenados, num_act_min = 2, num_act_max = 20)
fitness_mejor_individuo = 0
generaciones_sin_mejora = 0
generacion = 0

while generaciones_sin_mejora < 100:

    generacion += 1

    # Calculamos la función de fitness a la generación actual
    lista_fitness = list()
    lista_pesos_carteras_fitness =  list()
    for cartera in range(len(lista_posiciones)):

        fitness, pesos_fitness = calcular_fitness(datos_ordenados = datos_ordenados, posiciones = lista_posiciones[cartera])
        lista_fitness.append(fitness)
        lista_pesos_carteras_fitness.append(pesos_fitness)

    # Guardamos el mejor individuo de la generación actual
    if fitness_mejor_individuo < max(lista_fitness):

        fitness_mejor_individuo = max(lista_fitness)            
        posicion_mejor_individuo = np.where(lista_fitness == fitness_mejor_individuo)
        pesos_cartera_mejor_individuo = lista_pesos_carteras_fitness[posicion_mejor_individuo[0][0]]
        activos_cartera_mejor_individuo = lista_posiciones[posicion_mejor_individuo[0][0]]
        print(f'Generación {generacion}. El mejor individuo tiene una eficiencia de {fitness_mejor_individuo}')
        generaciones_sin_mejora = 0

    else:

        print(f'Generación {generacion+1}. Sin mejora generacional respecto a la anterior')
        generaciones_sin_mejora += 1

    # Llevamos a cabo la selección de padres, cruzamiento, mutación y matriz de descendencia
    lista_posiciones, fitness_mejor_individuo = reemplazo_generacional(datos_ordenados, lista_posiciones, lista_fitness, lista_pesos_carteras_fitness, activos_cartera_mejor_individuo)

print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)    

Generación 1. El mejor individuo tiene una eficiencia de 0.2965968649238079
Generación 2. El mejor individuo tiene una eficiencia de 0.4284376549378159
Generación 3. El mejor individuo tiene una eficiencia de 0.5162507920469959
Generación 4. El mejor individuo tiene una eficiencia de 0.541918928215972
Generación 5. El mejor individuo tiene una eficiencia de 0.5824180422868008
Generación 6. El mejor individuo tiene una eficiencia de 0.5942747504348346
Generación 7. El mejor individuo tiene una eficiencia de 0.6820649336696968
Generación 8. El mejor individuo tiene una eficiencia de 0.6883560208265168
Generación 9. El mejor individuo tiene una eficiencia de 0.7493390157983095
Generación 10. El mejor individuo tiene una eficiencia de 0.7517847185596332
Generación 11. El mejor individuo tiene una eficiencia de 0.7574855026744427
Generación 13. Sin mejora generacional respecto a la anterior
Generación 14. Sin mejora generacional respecto a la anterior
Generación 15. Sin mejora generacional 

In [25]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.5064945990483187
[   95  4664  7340 16218 21864 39070 42607]
[[[0.17109895 0.07464691 0.         0.         0.41191771 0.232657
   0.10967943]]]


In [27]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.5064176260284787
[   94  4664 21864 39093 39177 50782]
[[[0.16753853 0.09455427 0.14516729 0.10227152 0.26882037 0.22164802]]]


In [29]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.5073564829472692
[   95  4234  4664  5193 21240 39070 47601]
[[[0.2903647  0.         0.0613204  0.         0.         0.55423573
   0.09407916]]]
